### For Table of Contents: 
#### https://github.com/minrk/ipython_extensions

```
curl -L https://rawgithub.com/minrk/ipython_extensions/master/nbextensions/toc.js > $(ipython locate)/nbextensions/toc.js
curl -L https://rawgithub.com/minrk/ipython_extensions/master/nbextensions/toc.css > $(ipython locate)/nbextensions/toc.css

echo '
$([IPython.events]).on("app_initialized.NotebookApp", function () {
    IPython.load_extensions("toc");
});' >> $(ipython locate)/profile_default/static/custom/custom.js

```

# GPDB Demo Setup

### Note:  This notebook is always evolving.  It will be updated periodically to include nifty tips/tricks.

1. Create a directory ```/home/gpadmin/bds``` on the master node.
2. Put the following files (found on Google drive) into ```/home/gpadmin/bds```:
  - https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
  - crimes-2001-present.csv.tar.gz (unzip via ```tar -xvf crimes-2001-present.csv.tar.gz```)
  - ```cp crimes-2001-present.csv crimes-all.csv```
  - gpdemo-dwbi.sql
  - resource_queue (uzip via `tar -xvf resource_queues.tar`)
  - weather folder (contains several .csv files for loading)
3. Use Appendix A of this script to create two custom views that show storage and compression data.  
4. Use Appendix B of this script to create functions that will be used in this script
5. Use Appendix C of this script to make this dataset larger.  Recommend 50M records for demo on DCA.
6. ~~Use a tool like pgAdmin3 or DbVisualizer to execute the statements in this script during the demo.~~

In [ ]:
"""
-- Connect to the dca and database through a terminal window.
ssh gpadmin@10.68.128.7  --supply password if prompted (changeme)
gpstate; gpstate -e; gpstate -m  -- check the status of the database and the mirrors
""";

"""
-- 1. Using the commmand line.
psql -d postgres; 
\l  -- list databases
create database bds; \c bds;
\d  -- list tables
\?, \h, \h vacuum

""";

#### Install ipython-sql:
```
    pip install ipython-sql
```

#### Adjust path to allow psycopg2 to install:
```
    PATH=$PATH:/path/to/pg_config pip install psycopg2 pip install psycopg2
```

#### may need to do this also:
``` 
    install_name_tool -change libcrypto.1.0.0.dylib \
    /Users/kdunn/anaconda/lib/libcrypto.1.0.0.dylib \
    /Users/kdunn/anaconda/lib/python2.7/site-packages/psycopg2/_psycopg.so
```

#### and the same for libssl.1.0.0.dylib:
``` 
    install_name_tool -change libssl.1.0.0.dylib \
    /Users/kdunn/anaconda/lib/libssl.1.0.0.dylib \
    /Users/kdunn/anaconda/lib/python2.7/site-packages/psycopg2/_psycopg.so
```

Reference: https://github.com/catherinedevlin/ipython-sql

In [10]:
%load_ext sql
%sql postgresql://gpadmin@192.168.69.145/gpadmin

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: gpadmin@gpadmin'

# Part I: Loading Data

### 2. Create the table to hold the crimes data from data.gov.  we already know the layout.

In [13]:
%%sql 
drop table if exists crimes;
CREATE TABLE crimes
(
  id int
 ,case_number varchar (20)
 ,crime_date timestamp
 ,block varchar(50)
 ,IUCR varchar(10)
 ,primary_type varchar(50)
 ,description varchar(75)
 ,location_desc varchar (75)
 ,arrest boolean
 ,domestic boolean
 ,beat varchar(7)
 ,district varchar(7)
 ,ward smallint
 ,community_area varchar(10)
 ,fbi_code varchar(5)
 ,x_coord float
 ,y_coord float
 ,crime_year smallint
 ,record_update_date timestamp
 ,latitude float
 ,longitude float
 ,location varchar (60)
)
distributed by (id);

drop table if exists weather_data;
CREATE TABLE weather_data
(
  CST timestamp
  ,MaxTemperatureF int
  ,MeanTemperature int
  ,MinTemperatureF int
  ,MaxDewPointF float
  ,MeanDewPointF float
  ,MinDewpointF float
  ,MaxHumidity float
  ,MeanHumidity float
  ,MinHumidity float
  ,MaxSeaLevelPressureIn float
  ,MeanSeaLevelPressureIn float
  ,MinSeaLevelPressureIn float
  ,MaxVisibilityMiles float
  ,MeanVisibilityMiles float
  ,MinVisibilityMiles float
  ,MaxWindSpeedMPH float
  ,MeanWindSpeedMPH float
  ,MaxGustSpeedMPH float
  ,PrecipitationIn varchar(20)
  , CloudCover float
  , Events varchar(50)
  , WindDirDegrees float);

Done.
Done.
Done.
Done.
291058 rows affected.
1 rows affected.
0 rows affected.
Done.


[]

## 4. Use the 'COPY' command in the command line to bulk-load data via the master server.
timing on (note, psycopg2 isn't a fan the timing option')

In [ ]:
%%sql

COPY crimes FROM '/home/gpadmin/chicagoCrimes-all-2001-July262015.csv' 
CSV HEADER LOG ERRORS INTO err_crimes KEEP SEGMENT REJECT LIMIT 50 ROWS;  -- (13 secs.)

select count(*) total_records from crimes;
select * from err_crimes;

truncate table crimes;

## 5. Parallel loading with gpfdist  
### Kill and restart the gpfdist utility on the database


```
ps ax | grep gpfdist
pkill -9 gpfdist
gpfdist -d /home/gpadmin/bds/ -p 8081 -l /home/gpadmin/bds/gpfdist.log &
```

### Create an external table that 'points' to the source file.
NOTE:  can load multiple zipped files in parallel without unzipping.  gpfdist://mdw:8081/*.gz'   VERY FAST!) 

In [ ]:
%%sql

drop external table if exists ext_crimes;
drop external table if exists ext_weather_data;

create external table ext_crimes (like crimes) 
location ('gpfdist://mdw:8081/crimes_all.csv') 
format 'csv' (header);

create external table ext_weather_data (like weather_data) 
location ('gpfdist://mdw:8081/weather/*.csv') 
format 'csv' (header);

### View the data to see there are no tricks.

In [ ]:
%%sql

select count(*) total_recs from crimes;
select count(*) total_recs from weather_data;

-- (35 secs, only works if the file is unzipped)
select count(*) total_recs from ext_crimes;  

select count(*) total_recs from ext_weather_data;

### Load the data from the source file (external table) into the database table.

In [ ]:
%%sql

-- (60M recs, 45 seconds on 1/4 rack DCA; 5 minutes on a single-node VM)
insert into crimes (select * from ext_crimes);  

insert into weather_data (select * from ext_weather_data);
select count(*) total_recs from crimes;
select count(*) total_recs from weather_data;
select * from crimes limit 10;
select * from weather_data order by 1 desc limit 100;
select distinct primary_type from crimes;
select distinct events from weather_data;

### we'll do a bit more stress-testing down below

## 6. Data load from external web source - Worldwide earthquakes for the last 7 days  
https://explore.data.gov/Geography-and-Environment/Worldwide-M1-Earthquakes-Past-7-Days/7tag-iwnu


In [14]:
%%sql

DROP TABLE IF EXISTS wwearthquakes_lastwk;
CREATE TABLE wwearthquakes_lastwk (
    time TEXT, 
    latitude numeric, 
    longitude numeric, 
    depth numeric, 
    mag numeric, 
    mag_type varchar (10),
    NST integer, 
    gap numeric, 
    dmin numeric, 
    rms text, 
    net text, 
    id text, 
    updated TEXT, 
    place varchar(150), 
    type varchar(50)
)
DISTRIBUTED BY (time);

DROP EXTERNAL TABLE IF EXISTS ext_wwearthquakes_lastwk;

create external web table ext_wwearthquakes_lastwk (like wwearthquakes_lastwk) 
-- defining an OS command to execute
Execute 'wget -qO - http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.csv' 
ON MASTER
Format 'CSV' (HEADER)
Segment Reject limit 300;

Done.
Done.
Done.
Done.


[]

### Load and browse the data into the table

In [ ]:
%%sql

insert into wwearthquakes_lastwk select * from ext_wwearthquakes_lastwk;
select count(*) total_records from wwearthquakes_lastwk;
select * from wwearthquakes_lastwk order by 1 desc limit 100;

## 7. Using a Sequence during load.  
### Note the 'blocks of values' assigned;  the master is the single source of truth.

In [ ]:
%%sql 

drop sequence if exists myseq;
create sequence myseq start 300 cache 1000;

drop table if exists crimes_seq;
create table crimes_seq as (select nextval('myseq') 
                            as my_key, id, ward, primary_type 
                            from crimes limit 100);

select * from crimes_seq order by 1;

## 8. Test pl/pgsql functions.  
### Use Appendix B or `psql -f /home/gpadmin/bds/xxx.sql` to build them

In [ ]:
%%sql

select buildDateDim (date '2001-01-01', date '2014-12-31'); 
select * from dim_date order by 1 limit 100;
select buildTimeDim(); select * from dim_time order by 1;
select myfunc (3, 'Hello');

## 9. Best practice to do `VACUUM & ANALYZE` routinely and after big loads

In [ ]:
%%sql

vacuum analyze crimes;  
vacuum analyze weather_data;

# Part II - Data Distribution, Partitioning, and Polymorphic Storage

### 1. Create a table with a bad distribution key.

In [ ]:
%%sql

drop table if exists crimes_bad_key;
create table crimes_bad_key (like crimes) distributed by (arrest);

### Load data into the table and browse.

In [18]:
%%sql

-- 175 seconds on a 1/4 rack
insert into crimes_bad_key (select * from crimes);  
select count(*) total_records from crimes_bad_key;

Done.
Done.
0 rows affected.
1 rows affected.
(psycopg2.ProgrammingError) schema "gp_toolkit" does not exist
LINE 5: select * from gp_toolkit.gp_skew_coefficients ;
                      ^
 [SQL: "-- View skew.  Could also be done live in Command Center in real time.\n-- Look at 'gp_skew_coefficients' in gp_toolkit.  Lower number is better.\n-- Look at 'gp_skew_idle_fractions' in gp_toolkit.  0.1 = 10%% idle, which is ok.  0.5 = 50%%, which is bad.\n\nselect * from gp_toolkit.gp_skew_coefficients ;"]


#### View skew.  Could also be done live in Command Center in real time.
#### Look at 'gp_skew_coefficients' in gp_toolkit.  Lower number is better.
#### Look at 'gp_skew_idle_fractions' in gp_toolkit.  0.1 = 10% idle, which is ok.  0.5 = 50%, which is bad
#### TODO (doesn't work in HAWQ) select * from gp_toolkit.gp_skew_coefficients ;

### A binary key will result in highly skewed data load (compare to crimes)

In [20]:
%%sql

select distinct (arrest) from crimes;

0 rows affected.


arrest


### 2. Create a partitioned table

In [22]:
%%sql 

drop table if exists crimes_part;
create table crimes_part(
  id int
 ,case_number varchar (20)
 ,crime_date timestamp
 ,block varchar(50)
 ,IUCR varchar(10)
 ,primary_type varchar(50)
 ,description varchar(75)
 ,location_desc varchar (75)
 ,arrest boolean
 ,domestic boolean
 ,beat varchar(7)
 ,district varchar(7)
 ,ward smallint
 ,community_area varchar(10)
 ,fbi_code varchar(5)
 ,x_coord float
 ,y_coord float
 ,crime_year smallint
 ,record_update_date timestamp
 ,latitude float
 ,longitude float
 ,location varchar (60)
)

distributed by (id)
partition by list (primary_type) 
    (partition p1 values ('ROBBERY'), 
     partition p2 values ('BURGLARY'), 
     partition p3 values ('THEFT'), 
     partition p4 values ('PROSTITUTION'),
     partition p5 values ('RITUALISM'), 
     partition p6 values ('NARCOTICS'), 
     partition p7 values ('KIDNAPPING'), 
     partition p8 values ('INTIMIDATION'),
     partition p9 values ('BATTERY'), 
     partition p10 values ('ASSAULT'), 
     partition p11 values ('ARSON'), 
     partition p12 values ('STALKING'),
     partition p13 values ('GAMBLING'), 
     partition p14 values ('DOMESTIC VIOLENCE'), 
     partition p15 values ('WEAPONS VIOLATION'), 
     partition p16 values ('DECEPTIVE PRACTICE'),
     default partition other );

Done.
Done.


[]

## Load data into the partitioned table and browse the counts.

In [23]:
%%sql 

insert into crimes_part (select * from crimes);  -- (15 secs.)
select count(*) total_records from crimes_part;

0 rows affected.
1 rows affected.


total_records
0


## 3. Generate an explain plan
### See query cost on the non-partitioned table

In [24]:
%%sql

explain
select crimes.*
from crimes
where primary_type in ('NARCOTICS', 'GAMBLING')
order by id;  -- (3.4 secs)

5 rows affected.


QUERY PLAN
Sort (cost=0.00..3.68 rows=1 width=173)
Sort Key: id
-> Gather Motion 1:1 (slice1; segments: 1) (cost=0.00..2.68 rows=1 width=173)
-> Table Scan on crimes (cost=0.00..1.51 rows=1 width=173)
"Filter: primary_type::text = ANY (ARRAY['NARCOTICS'::character varying, 'GAMBLING'::character varying]::text[])"


### Generate an explain plan to see the cost of the query on the partitioned table

In [25]:
%%sql

explain
select crimes_part.*
from crimes_part
where primary_type in ('NARCOTICS', 'GAMBLING')
order by id; -- (1 sec.)

8 rows affected.


QUERY PLAN
Sort (cost=0.00..5.03 rows=1 width=176)
Sort Key: crimes_part.id
-> Gather Motion 1:1 (slice1; segments: 1) (cost=0.00..4.03 rows=1 width=176)
-> Sequence (cost=0.00..2.86 rows=1 width=176)
-> Result (cost=0.00..1.52 rows=1 width=176)
-> Function Scan on gp_partition_expansion (cost=10.00..100.00 rows=100 width=4)
-> Dynamic Table Scan on crimes_part (partIndex: 0) (cost=0.00..2.86 rows=1 width=176)
"Filter: primary_type::text = ANY (ARRAY['NARCOTICS'::character varying, 'GAMBLING'::character varying]::text[])"


## 4. Storage and Compression.  
### Create the table that's column-oriented with quickLZ compression

In [ ]:
%%sql

drop table if exists crimes_qlz;
create table crimes_qlz (like crimes)
with (appendonly=true, orientation=column, compresstype=quicklz)
partition by list (primary_type) 
    (partition p1 values ('ROBBERY'), 
     partition p2 values ('BURGLARY'), 
     partition p3 values ('THEFT'), 
     partition p4 values ('PROSTITUTION'),
     partition p5 values ('RITUALISM'), 
     partition p6 values ('NARCOTICS'), 
     partition p7 values ('KIDNAPPING'), 
     partition p8 values ('INTIMIDATION'),
     partition p9 values ('BATTERY'), 
     partition p10 values ('ASSAULT'), 
     partition p11 values ('ARSON'), 
     partition p12 values ('STALKING'),
     partition p13 values ('GAMBLING'), 
     partition p14 values ('DOMESTIC VIOLENCE'), 
     partition p15 values ('WEAPONS VIOLATION'), 
     partition p16 values ('DECEPTIVE PRACTICE'),
     default partition other );

### Load the data

In [ ]:
%%sql

insert into crimes_qlz (select * from crimes);  --12 secs.
select count(*) total_records from crimes_qlz;

### Look at the explain plan

In [ ]:
%%sql

explain
select crimes_qlz.*
from crimes_qlz
where primary_type in ('NARCOTICS', 'GAMBLING')
order by id; -- (1 sec.)

### Create a table with zLib, level=5

In [ ]:
%%sql

drop table if exists crimes_zlib;
create table crimes_zlib (like crimes)
with (appendonly=true, orientation=column, compresstype=zlib, compresslevel=5)
partition by list (primary_type) 
    (partition p1 values ('ROBBERY'), 
     partition p2 values ('BURGLARY'), 
     partition p3 values ('THEFT'), 
     partition p4 values ('PROSTITUTION'),
     partition p5 values ('RITUALISM'), 
     partition p6 values ('NARCOTICS'), 
     partition p7 values ('KIDNAPPING'), 
     partition p8 values ('INTIMIDATION'),
     partition p9 values ('BATTERY'), 
     partition p10 values ('ASSAULT'), 
     partition p11 values ('ARSON'), 
     partition p12 values ('STALKING'),
     partition p13 values ('GAMBLING'), 
     partition p14 values ('DOMESTIC VIOLENCE'), 
     partition p15 values ('WEAPONS VIOLATION'), 
     partition p16 values ('DECEPTIVE PRACTICE'),
     default partition other );

### load the data

In [ ]:
%%sql

insert into crimes_zlib (select * from crimes);  --(25 secs.)
select count(*) as total_recs from crimes_zlib;

### Look at the explain plan

In [ ]:
%%sql

explain
select crimes_zlib.*
from crimes_zlib
where primary_type in ('NARCOTICS', 'GAMBLING')
order by id; -- (1 sec.)

## 5. With all the same counts, look at the size of tables  
### Note:  No Indexes!!!

In [ ]:
%%sql

SELECT tabs.nspname AS schema_name
,      COALESCE(parts.tablename, tabs.relname) AS table_name
,      ROUND(SUM(sotaidtablesize)/1024/1024/1024,3) AS table_GB
,      ROUND(SUM(sotaididxsize)/1024/1024/1024,3) AS index_GB
FROM   gp_toolkit.gp_size_of_table_and_indexes_disk sotd
,     (SELECT c.oid, c.relname, n.nspname
       FROM   pg_class c
       ,      pg_namespace n
       WHERE  n.oid = c.relnamespace
       AND    c.relname NOT LIKE '%_err'
      )tabs
LEFT JOIN pg_partitions parts
ON     tabs.nspname = parts.schemaname
AND    tabs.relname = parts.partitiontablename
WHERE  sotd.sotaidoid = tabs.oid and tabs.nspname = 'public'
GROUP BY tabs.nspname, COALESCE(parts.tablename, tabs.relname)
ORDER BY 1 desc,2;


## 6. View this customized view for table, storage, and compression characteristics

In [ ]:
%%sql 

select * from v_gp_table_storage2;

## 7. Swapping partitions  
### Create a table that represents one of the partitions and load it

In [ ]:
%%sql

drop table if exists crimes_p0;
create table crimes_p0 (like crimes_part) 
with (appendonly=true, compresstype=quicklz, orientation=column);

insert into crimes_p0 (select * from crimes where primary_type = 'NARCOTICS');
select count(*) total_records from crimes_p0;

-- swap the partition in the table with our 'latest' table.  
-- Note in pdAdmin3 the table layout.  The partition is now different from others.
alter table crimes_part EXCHANGE PARTITION FOR ('NARCOTICS') WITH TABLE crimes_p0;
select count(*) total_records from crimes_part where primary_type = 'NARCOTICS';

## 8. (Optional) Perform garbage-collection and cleanup

In [ ]:
%%sql

vacuum analyze crimes;
vacuum analyze crimes_part;
vacuum analyze crimes_qlz;
vacuum analyze crimes_zlib;
vacuum analyze weather_data;

# Part III BI/ETL Interoperability

### Pentaho integration video in gDrive: `gpdb-pentaho.mov`

- create/view a database connection.
- create/view a new transformation mapping
- source = .csv (same as before); target = gpdb
- execute the ETL transformation via sh spoon.sh --file xxx.ktr
- note:  it creates the YAML file and .dat file for gpload.  can also have it just create the YAML and .dat file w/o executing
- more crimes.yaml; more crimes.dat
- call the gpload utility (gpload -f filename.cfg) to load the data

In [ ]:
%%sql 

select count(*) total_recs from cms;

### TODO: CONNECT GPDB TO HDFS VIA GPHDFS PROTOCOL

# Part IV - Advanced SQL and Analytics
### Windowing Functions and Analytics via Madlib

#### Stress test

In [ ]:
%%sql

select
  to_char(a.crime_date, 'yyyy')
  ,a.primary_type
  ,count(a.primary_type)
from crimes a
join (select to_char(b.crime_date, 'yyyy'), b.primary_type, count(b.primary_type)
      from crimes b
      where to_char(crime_date, 'yyyy') = to_char(now() - interval '1 year', 'yyyy')
      group by 1,2
      order by count(b.primary_type) desc
      limit 5) c on a.primary_type=c.primary_type
where to_char(crime_date, 'yyyy') in (to_char(now(), 'yyyy'), to_char(now() - interval '1 year', 'yyyy'))
group by 1,2
order by 2,1 desc;

#### Big Query

In [ ]:
%%sql

select primary_type, arrest, year_month, month_name, day_name, count(*)
from crimes a 
inner join dim_date b on to_char(a.crime_date, 'yyyy-mm-dd') = to_char(b.date_value, 'yyyy-mm-dd')
group by 1,2,3,4,5
limit 20;

#### Small Query

In [ ]:
%%sql

select primary_type, arrest, year_month, month_name, day_name, count(*)
from crimes_part a 
inner join dim_date b on to_char(a.crime_date, 'yyyy-mm-dd') = to_char(b.date_value, 'yyyy-mm-dd')
where a.primary_type = 'NARCOTICS'
group by 1,2,3,4,5
limit 20;

#### WINDOWING FUNCTION.  RUNNING COUNT OF CRIMES BY DISTRICT. ADJUST TO INCLUDE MONTH.

In [ ]:
%%sql

explain
select district, count(distinct id)  ,
       sum(count(distinct id)) over (order by district, count(distinct id) desc) as running_sum
from crimes
where district in ('003', '014', '031') 
group by district;

#### Summary Analytics

In [ ]:
%%sql

SELECT * FROM madlib.summary( 'bds.crimes', 
                              'bds.crimes_summary', 
                              'block,beat,ward,primary_type', 
                              'arrest', 
                              TRUE, 
                              TRUE, 
                              NULL, 
                              5, 
                              FALSE
                            );
select * from crimes_summary;

#### Linear Regression: 
(need to train a model, then run a 2nd time to determine what the dependent variables are.  Used for Prediction)

In [ ]:
%%sql

SELECT madlib.linregr_train( 'crimes',
                             'crimes_linregr',
                             'arrest_flag::int',
                             'ARRAY[1, block, ward, primary_type, beat]'
                           );

#### Correlation matrix between all the numeric columns in the table.

In [ ]:
select * from example_data;
SELECT madlib.correlation( 'example_data',
                           'example_data_output'
                         );

In [ ]:
select * from example_data_output order by column_position;

# Part V - Backup Options

### GPDUMP is a parallelized backup utility.
Usage:
```'gpcrondump -x ..'  ```

### PGDUMP is a utility for backing up data when you know it'll be restored to a different number of segments.
```pg_dump -t crimes_seq bds > backup1.sql;```  
- utility creates one file on the server.  Loaded via COPY.

```pg_dump --column-inserts -t crimes_seq bds > backup2.sql``` 
- option to include all insert statements.

```pg_restore``` is used to restore the database.  Has many options as well.



### Use a writable external tables for controlled backup.
#### Create an external table that we can write data to (but really its a file).

In [ ]:
%%sql

drop external table if exists crimes_export;
create writable external table crimes_export (like crimes)
location ('gpfdist://mdw:8081/crimes_backup.csv')
format 'csv' (delimiter ',' null '');

#### Load data into the external table (flat file).

In [ ]:
%%sql 

insert into crimes_export (select * from crimes where crime_year = 2014);

-- you get an error, but you can see the file in the directory.
select count(*) from crimes_export; 

#### Re-load the data from the backup.  Start with creating an external table.

In [ ]:
%%sql 

drop external table if exists crimes_backup;
create external table crimes_backup (like crimes)
location ('gpfdist://mdw:8081/crimes_backup.csv')
format 'csv' (delimiter ',' null '');

In [ ]:
%%sql

select count(*) total_records from crimes_backup;
select count(*) total_records from crimes;
insert into crimes (select * from crimes_backup);

# Part VI - Hadoop and Hawq
1. Perform loading of same data into Hadoop
  - hadoop fs -put
  - HUE
  - ETL Program
  - Internal/external tables in Hive & Hawq
2. SQL Capacbility (Hive vs. Hawq)
3. SQL Performance (Hive vs. Hawq vs. GPDB)
4. Attempt an update on a Hawq table
5. Store data in Parquet format

## Hive

In [ ]:
%sql <<hive uri>>

Note: Below will automatically create the directory in HDFS

In [2]:
%%sql

DROP TABLE crimes;
CREATE TABLE crimes (
     id string
    ,case_number string
    ,crime_date string
    ,block string
    ,IUCR string
    ,primary_type string
    ,description string
    ,location_desc string
    ,arrest_flag string
    ,domestic_flag string
    ,beat string
    ,district string
    ,ward string
    ,community_area string
    ,fbi_code string
    ,x_coordinate string
    ,y_coordinate string
    ,year string
    ,rec_date string
    ,latitude string
    ,longitude string
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
STORED AS TEXTFILE
LOCATION '/bmg/crimes/';  

### Load data into Hive (from Hive)

In [ ]:
%%sql

--(alternative to hadoop fs -put ...)
LOAD DATA LOCAL INPATH '/home/gpadmin/bds/crimes.csv.tar.gz' 
OVERWRITE INTO TABLE crimes;

select * from crimes limit 1;
select count(*) from crimes;

select year, description, count(*) total_crimes
from crimes
where year in ('2011', '2012') and primary_type in ('CRIMINAL DAMAGE')
group by year, description
order by year, description
limit 30;

### Can't support ANSI-Standard SQL

In [ ]:
%%sql

select year, description, count(*) total_crimes
from crimes
group by 1,2
order by 3, 1 desc
limit 30;

with a as (select count(*) from crimes)
select * from a;

select * from crimes where primary_type not in 
(select primary_type from crimes where primary_type in ('THEFT'));

## Hawq

In [ ]:
%sql postgresql://gpadmin@localhost/hawqDb

In [ ]:
%%sql

drop table if exists crimes;
CREATE TABLE crimes (
     id varchar (15)
    ,case_number varchar(50)
    ,crime_date varchar(50)
    ,block varchar(75)
    ,IUCR varchar(75)
    ,primary_type varchar(75)
    ,description varchar(100)
    ,location_desc varchar(125)
    ,arrest_flag varchar(20)
    ,domestic_flag varchar(20)
    ,beat varchar(15)
    ,district varchar(15)
    ,ward varchar(15)
    ,community_area varchar(15)
    ,fbi_code varchar(15)
    ,x_coordinate varchar(20)
    ,y_coordinate varchar(20)
    ,year char(4)
    ,rec_date varchar(20)
    ,latitude varchar(20)
    ,longitude varchar(20)
) 
DISTRIBUTED by (id);

rejects errors, does not put file into hdfs directory

(rejected errors + loaded records) has a difference of 3 between hawq and hive.

```
zcat crimes.csv.tar.gz | psql -c "COPY crimes FROM STDIN CSV DELIMITER ',' SEGMENT REJECT LIMIT 100000;"
```

### ANSI SQL Capabilities

In [ ]:
%%sql

select year, description, count(*) total_crimes
from crimes
where year in ('2011', '2012') and primary_type in ('CRIMINAL DAMAGE')
group by 1,2
order by 1,2
limit 30;

### sub-select

In [ ]:
%%sql

select * from crimes 
where primary_type not in (select primary_type from crimes where primary_type in ('THEFT')) 
limit 5;

### window functions

In [ ]:
%%sql

explain
select district, count(distinct ward)  ,
       sum(count(distinct ward)) over (order by district, count(distinct ward) desc) 
as running_sum
from crimes
where district in ('001', '014', '031') 
group by district;

### sequences

In [ ]:
%%sql

drop sequence if exists myseq;
create sequence myseq start 1 increment by 1 cache 1000;

drop table if exists seq_crimes;
create table seq_crimes as 
(select nextval('myseq'), year, description, count(*) total_crimes
from crimes
group by 2,3
order by 1,2,3 desc
limit 20);

select * from seq_crimes order by 1;

### Partitioning

In [ ]:
%%sql

drop table if exists crimes_part;
create table crimes_part (like crimes)
distributed by (id)
partition by list (primary_type)
    (partition p1 values ('ROBBERY', 'BURGLARY'), 
     partition p2 values ('HOMOCIDE'), 
     default partition other);

insert into crimes_part (select * from crimes);

explain
select a.*
--from crimes a
from crimes_part a
where primary_type in ('HOMICIDE');

### query to see table sizes

In [ ]:
%%sql

SELECT nspname || '.' || relname AS "relation", pg_size_pretty(pg_relation_size(C.oid)) 
AS "size" 
FROM pg_class C  
LEFT JOIN pg_namespace N ON (N.oid = C.relnamespace) 
WHERE nspname IN ('public', 'retail_demo') 
ORDER BY pg_relation_size(C.oid) DESC LIMIT 20;

## Connect a BI tool to HAWQ

### http://nbviewer.ipython.org/gist/catherinedevlin/6588378

# Appendix A: Creating Custom Views for Table Storage

### Custom view for table storage

In [ ]:
%% sql

CREATE OR REPLACE VIEW public.v_gp_table_storage AS
SELECT current_timestamp AS tms, n.nspname AS schema_name, c.relname AS table_name,
        CASE
            WHEN c.relstorage = 'a'::"char" THEN 'row append-only'::text
            WHEN c.relstorage = 'c'::"char" THEN 'column append-only'::text
            WHEN c.relstorage = 'h'::"char" THEN 'heap'::text
            WHEN c.relstorage = 'x'::"char" THEN 'external'::text
            ELSE NULL::text
        END AS storage_type,
              a.compresstype  AS compr_type,
              a.compresslevel AS compr_level,
              sotailtablesizedisk                                   as tabind_sz,
              (sotailtablesizedisk         / 1024^3)::numeric(20,2) as tabind_sz_gb,
              (sotailtablesizeuncompressed / 1024^3)::numeric(20,2) as tabind_sz_unc_gb,
              case WHEN (sotailtablesizedisk=0 or sotailtablesizedisk is null) THEN -1 ELSE (sotailtablesizeuncompressed/sotailtablesizedisk)::numeric(6,1) END as compr_ratio
              , c.relhassubclass as is_partitioned
   FROM pg_class c
   LEFT JOIN pg_appendonly a ON c.oid = a.relid
   LEFT JOIN pg_namespace n ON n.oid = c.relnamespace
   LEFT JOIN gp_toolkit.gp_size_of_table_and_indexes_licensing sot ON sot.sotailoid = c.oid
  WHERE (n.nspname <> ALL (ARRAY['information_schema'::name, 'pg_catalog'::name, 'pg_toast'::name, 'gp_toolkit'::name])) AND c.relkind = 'r'::"char"
;

### Another custom view for table storage.  Requires the first custom view

In [ ]:
%%sql

CREATE OR REPLACE VIEW public.v_gp_table_storage2 AS
 SELECT tms,
        schema_name,
        regexp_replace(table_name::text, '_1_prt_.*$'::text, ''::text) AS table_name,
              storage_type,
              compr_type,
              compr_level,
              count(*)                            AS nr_of_partitions,
              sum(tabind_sz)                      AS tabind_size,
              sum(tabind_sz_gb)                   AS tabind_sz_gb,
              sum(tabind_sz_unc_gb)               AS tabind_sz_unc_gb,
              round(avg(compr_ratio)::numeric, 2) AS avg_compr_ratio
   FROM public.v_gp_table_storage
  WHERE storage_type <> 'external'::text AND table_name !~~ 'err_%'::text AND not is_partitioned
  AND (COMPR_TYPE IS NOT NULL OR TABLE_NAME='crimes')
  GROUP BY tms, schema_name, regexp_replace(table_name::text, '_1_prt_.*$'::text, ''::text), storage_type, compr_type, compr_level
  ORDER BY 3;

# Appendix B: Simple PL/PSQL Functions

### Function that will apply logic based on user inputfile

In [ ]:
%%sql


CREATE OR REPLACE FUNCTION myFunc (numtimes integer, msg text)
  RETURNS text AS
$BODY$
DECLARE
    strresult text;
BEGIN
    strresult := '';
    IF numtimes = 1 THEN
        strresult := 'Only one row!';
    ELSIF numtimes > 0 AND numtimes < 11 THEN
        FOR i IN 1 .. numtimes LOOP
            strresult := strresult || msg || '; '; --E'\r\n';
        END LOOP;
    ELSE
        strresult := 'You can not do that.';
        IF numtimes <= 0 THEN
            strresult := strresult || ' Must be greater than zero.';
        ELSIF numtimes > 10 THEN
            strresult := strresult || ' That''s too many items!';
        END IF;
    END IF;
    RETURN strresult;
END;
$BODY$
  LANGUAGE 'plpgsql' IMMUTABLE;
ALTER FUNCTION myFunc(integer, text) OWNER TO gpadmin;

### Function that will build a date dimension when called with start and end dates

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION builddatedim(startdate date, enddate date)
  RETURNS text AS
$BODY$
DECLARE
    strresult text default ('Table Created!');
    loopDate date default (startDate);
BEGIN 
  execute 'drop table if exists dim_date';
  execute 'create table dim_date (
      date_key integer 
    , date_year smallint
    , quarter_name char(2)
    , year_quarter char(7)
    , month_name varchar(15)
    , month_name_short char(3)
    , month_number smallint
    , year_month varchar(7)
    , date_value date
    , day_name varchar(11)
    , day_of_week_num smallint
    , day_of_year smallint
    , week_of_year smallint
    , day_type varchar(10)
    , date_formal varchar(20)
    , last_30days_ind smallint
    , last_60days_ind smallint
    , last_90days_ind smallint
    , rec_update_date date
    ) distributed by (date_key)';
  while loopDate <= endDate loop
    execute 'insert into dim_date values (' ||
             to_number(to_char(loopDate, 'yyyymmdd'), 99999999) || ', ' ||
             extract(year from loopDate) || ', ' ||
             '''Q' || extract(quarter from loopDate) || ''', ' ||
             '''' || rtrim(to_char(loopDate, 'yyyy')) || '-Q' || extract(quarter from loopDate) || ''', ' ||
             '''' || rtrim(to_char(loopDate, 'Month')) || ''', ' ||
             '''' || rtrim(to_char(loopDate, 'MON')) || ''', ' ||
             to_number(to_char(loopDate, 'MM'), 99) || ', ' ||
             '''' || rtrim(to_char(loopDate, 'yyyy-MM')) || ''', ' ||
             '''' || loopDate || ''', ' ||
             '''' || rtrim(to_char(loopDate, 'Day')) || ''', ' ||
             extract(isodow from loopDate) || ', ' ||
             to_number(to_char(loopDate, 'DDD'), 999) || ', ' ||
             to_number(to_char(loopDate, 'WW'), 99) || ', ' ||
             '''' || case when 
                       rtrim(to_char(loopDate, 'Day')) in ('Saturday', 'Sunday') then 'Weekend' else 'Weekday' 
                     end || ''', ' ||
             '''' || rtrim(to_char(loopDate, 'FMMonth FMDDth, yyyy')) || ''', ' ||
             case when 
               loopDate between (now()::date - interval '30 day') and now()::date then 1 else 0 
             end || ', ' ||
             case when 
               loopDate between (now()::date - interval '60 day') and now()::date then 1 else 0 
             end || ', ' ||
             case when 
               loopDate between (now()::date - interval '90 day') and now()::date then 1 else 0 
             end || ', ' ||
             '''' || now()::date || '''' ||
             ')';
    loopDate := loopDate + 1;

  end loop;
  RETURN strresult;
END;
$BODY$
  LANGUAGE plpgsql VOLATILE;
ALTER FUNCTION builddatedim(date, date)
  OWNER TO gpadmin;

### Function that will build a time dimension when called with start and end dates

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION buildtimedim()
  RETURNS text AS
$BODY$
DECLARE
    strresult text default ('Table Created!');
    loopTime time default ('0:00');
BEGIN 
  execute 'drop table if exists dim_time';
  execute 'create table dim_time (
               time_key char(4)
             , hour_minute time 
             , time_ampm varchar(8) 
             , hour_range_text varchar(20)
             , day_segment varchar(20)
            ) distributed by (time_key)';
  for i in 0..1439 loop
    execute 'insert into dim_time values (' ||
             '''' || to_char(loopTime, 'hh24mi') || ''', ' ||
             '''' || to_char(loopTime, 'hh24:mi') || ''', ' ||
             '''' || to_char(loopTime, 'hh:miam') || ''', ' ||
             '''' || case rtrim(to_char(loopTime, 'hh24')) 
                       when '00' then '12am-12:59am' when '01' then '1am-1:59am' when '02' then '2am-2:59am' 
                       when '03' then '3am-3:59am' when '04' then '4am-4:59am' when '05' then '5am-5:59am'
                       when '06' then '6am-6:59am' when '07' then '7am-7:59am' when '08' then '8am-8:59am'
                       when '09' then '9am-9:59am' when '10' then '10am-10:59am' when '11' then '11am-11:59am'
                       when '12' then '12pm-12:59pm' when '13' then '1pm-1:59pm' when '14' then '2pm-2:59pm' 
                       when '15' then '3pm-3:59pm' when '16' then '4pm-4:59pm' when '17' then '5pm-5:59pm'
                       when '18' then '6pm-6:59pm' when '19' then '7pm-7:59pm' when '20' then '8pm-8:59pm'
                       when '21' then '9pm-9:59pm' when '22' then '10pm-10:59pm' when '23' then '11pm-11:59pm'
                       else 'other' end || ''', ' ||
             '''' || case  
                       when rtrim(to_char(loopTime, 'hh24')) between  '00' and '04' then 'Late Night'
                       when rtrim(to_char(loopTime, 'hh24')) between  '05' and '07' then 'Early Morning'
                       when rtrim(to_char(loopTime, 'hh24')) between  '08' and '11' then 'Morning'
                       when rtrim(to_char(loopTime, 'hh24')) between  '12' and '16' then 'Afternoon'
                       when rtrim(to_char(loopTime, 'hh24')) between  '17' and '20' then 'Evening'
                     else 'Night' end || '''' ||
             ')';
    loopTime := loopTime + interval '1 minute';
  end loop;
  RETURN strresult;
END;
$BODY$
  LANGUAGE plpgsql VOLATILE;
ALTER FUNCTION buildtimedim()
  OWNER TO gpadmin;

# Appendix C:  Making the Crimes Data Larger

In [ ]:
%%sql

drop external table ext_crimes;
create external table ext_crimes (like crimes) 
location ('gpfdist://mdw:8081/crimes-2001-present.csv') 
format 'csv' (header);

-- NOTE:  EXECUTE THIS STATEMENT AS MANY TIMES AS NEEDED TO GET VOLUME REQUIRED.
insert into crimes (select * from ext_crimes);  
select count(*) total_records from crimes;

drop external table if exists crimes_export;
create writable external table crimes_export (like crimes)
location ('gpfdist://mdw:8081/crimes_all.csv')
format 'csv' (delimiter ',' null '');

insert into crimes_export (select * from crimes);

# Appendix D: Resource Queries

In [ ]:
%%sql

-- in gp_toolkit schema
-- gp_resq_priority_statement shows what's currently running
-- gp_locks_on_resqueue


-- QUEUE LIMITS
alter role user1 with resource queue none;
alter role user2 with resource queue none;
drop role if exists user1;
drop role if exists user2;

-- create resource queue with 2 active statements
drop resource queue q_activelimit;
create resource queue q_activelimit with (active_statements = 3);

-- create a user and assign them to a resource queue
create role user1 with login password 'user1';
alter user user1 with resource queue q_activelimit;
alter user user1 set search_path to public, gp_toolkit;
grant usage on schema public to user1;


-- cat small_query, big query, active1-3
-- active1.sh:  small query (7 secs)
-- active2.sh:  (5) small query (15 secs)
-- active3.sh   (10) small qeury (23 secs)
-- active4.sh   (5) small query, but only 3 run at once (check views)

--create resource queue based on cost.  Not used as frequently as its difficult to plan in ad hoc settings.
--drop resource queue q_costlimit;
--create resource queue q_costlimit with (max_cost=14e+6);  --14,000,000 cost limit

-- create a user and assign them to a resource queue
--create role user2 with login password 'user2';
--alter user user2 with resource queue q_costlimit;
--alter user user2 set search_path to public, gp_toolkit;
--grant usage on schema public to user2;

-- cost1.sh:    small query (7 secs) --show explain plan, only 1 can fit in the queue at once.
-- cost2.sh:    (3) small query (runs one at a time)
-- cost3.sh:    big query is beyond the cost limit, so you get error message

-- QUEUE PRIORITIES
create resource queue q_prioritymax with (active_statements=20, priority=max);
create resource queue q_prioritymed with (active_statements=20, priority=medium);
create resource queue q_prioritymin with (active_statements=20, priority=min);

create role usermax with login password 'usermax';
alter user usermax with resource queue q_prioritymax;
alter user usermax set search_path to public, gp_toolkit;
grant usage on schema public to usermax;

create role usermed with login password 'usermed';
alter user usermed with resource queue q_prioritymed;
alter user usermed set search_path to public, gp_toolkit;
grant usage on schema public to usermed;

create role usermin with login password 'usermin';
alter user usermin with resource queue q_prioritymin;
alter user usermin set search_path to public, gp_toolkit;
grant usage on schema public to usermin;

-- sh pri1.sh  -- notice same run-time on each run
-- sh pri2.sh  -- run same query on medium queue (note same timing regardless of queue)
-- sh pri3.sh  -- same on max queue

-- sh pri4.sh  -- run a mix of 3 med, 3 max;  max gets done faster; overall getting done faster (all small query)
-- sh pri5.sh  -- run slow query on min queue, then check CC.  then launch pri4 again. (times wont' be affected)




------


drop table if exists cms;
CREATE TABLE cms
(
  car_line_id character varying(20),
  bene_sex_ident_cd numeric(20),
  bene_age_cat_cd bigint,
  car_line_icd9_dgns_cd character varying(10),
  car_line_hcpcs_cd character varying(10),
  car_line_betos_cd character varying(5),
  car_line_srvc_cnt bigint,
  car_line_prvdr_type_cd bigint,
  car_line_cms_type_srvc_cd character varying(5),
  car_line_place_of_srvc_cd bigint,
  car_hcpcs_pmt_amt bigint
)
distributed by (car_line_id);